In [9]:
from collections import Counter
import pickle
import os

In [ ]:
def mtf_encode(text):  
    letters = ['0','1','2','3','4','5','6','7','8','9','$',' ']  
      
    res = ''  
      
    for i in range(len(text)):  
        mtf = letters.index(text[i])  
        if mtf<10:  
            res += str(mtf)  
        elif mtf == 10:  
            res += 'A'  
        elif mtf == 11:  
            res += 'B'  
        letters = letters[mtf:]+letters[:mtf]
    return res  

def mtf_decode(text):
    letters = ['0','1','2','3','4','5','6','7','8','9','$',' ']  
      
    res = ''  
      
    for char in text:  
        mtf = 0  
        if char == 'A':  
            mtf = 10  
        elif char == 'B':  
            mtf = 11  
        else:  
            mtf = int(char)  
        res += letters[mtf]  
        letters = letters[mtf:]+letters[:mtf]
    return res  

In [10]:
class Node:
    def __init__(self, value, left=None, right=None):
        self.right = right
        self.left = left
        self.value = value


def get_code(root, codes=dict(), code=''):
    if root is None:
        return

    if isinstance(root.value, str):
        codes[root.value] = code
        return codes

    get_code(root.left, codes, code + '0')
    get_code(root.right, codes, code + '1')

    return codes


def get_tree(string):
    string_count = Counter(string)

    if len(string_count) <= 1:
        node = Node(None)

        if len(string_count) == 1:
            node.left = Node([key for key in string_count][0])
            node.right = Node(None)

        string_count = {node: 1}

    while len(string_count) != 1:
        node = Node(None)
        spam = string_count.most_common()[:-3:-1]

        if isinstance(spam[0][0], str):
            node.left = Node(spam[0][0])

        else:
            node.left = spam[0][0]

        if isinstance(spam[1][0], str):
            node.right = Node(spam[1][0])

        else:
            node.right = spam[1][0]

        del string_count[spam[0][0]]
        del string_count[spam[1][0]]
        string_count[node] = spam[0][1] + spam[1][1]

    return [key for key in string_count][0]


def coding(string, codes):
    res = ''

    for symbol in string:
        res += codes[symbol]

    return res


def decoding(string, codes):
    res = ''
    i = 0

    while i < len(string):
        for code in codes:
            print(code)
            if string[i:].find(codes[code]) == 0:
                res += code
                i += len(codes[code])

    return res

In [11]:
def BWT(sequence):
    # Добавим символ конца строки 
    sequence += '$'
    # Получим список всех перестановок
    table = [sequence[index:] + sequence[:index] for index, _ in enumerate(sequence)]
    # Отсортируем этот список
    table.sort()
    # Возьмем последнюю букву строки из каждого элемента
    bwt = [rotation[-1] for rotation in table]
    # Это и есть результат кодирования алгоритмом bwt
    bwt = ''.join(bwt)

    return bwt

def inverse_BWT(sequence):
    # Разобьем строку на буквы
    table = [col for col in sequence]
    # Составим таблицу пройдя по всем элементам изначальной строки (первый элемент уже есть в нашей таблице)
    for i in range(len(sequence) - 1):
        # Сортируем элементы в таблице
        table.sort()
        # Склеиваем i-й элемент в таблице с i-м элементом в исходной строке
        table = [sequence[i] + table[i] for i in range(len(sequence))]
    # Результатом декодирования будет элемент, который заканчивается на символ конца строки
    return table[[row[-1] for row in table].index('$')] 

In [12]:
def compress(string):  
    bwt = BWT(string)
    mtf = mtf_encode(bwt)
    count_symbols = len(mtf)
    codes = get_code(get_tree(mtf))
    coding_str = coding(mtf, codes)
    
    return coding_str, codes, count_symbols

In [13]:
def decompress(string, code):
    hoff = decoding(string, code)
    mtf = mtf_decode(hoff)
    res = inverse_BWT(mtf)[:-1]
    
    return res

In [14]:
my_string = 'Строка для сжатия'
tree = get_tree(my_string)

codes = get_code(tree)
print(f'Шифр: {codes}')

coding_str = coding(my_string, codes)
print('Сжатая строка: ', coding_str)

def decoding(string, codes):
    
    # инвертируем словарь!
    codes2={}
    for key,value in codes.items():
        codes2[value] = key
    #print(codes2)
    
    code = ''
    res = ''
    for c in string:
        code = code + c
        if code in codes2:
            res = res+codes2[code]
            code = ''
    print(res)
    return res

res = decoding(coding_str, codes)

Шифр: {' ': '000', 'а': '001', 'и': '0100', 'ж': '0101', 'я': '011', 'д': '1000', 'к': '1001', 'с': '1010', 'л': '1011', 'т': '110', 'С': '1110', 'о': '11110', 'р': '11111'}
Сжатая строка:  11101101111111110100100100010001011011000101001010011100100011
Строка для сжатия


In [7]:
def zip_file(path, path_exit, L = 100):
    import os
    
    if not os.path.isfile(path):
        raise Exception('Файл не найден!')
        
    if not os.path.isdir(path_exit):
        raise Exception('Укажите путь для выгрузки результаты работы архиватора!')
    
    f = open(path, 'rb')
    rd = f.read()
    fd_str_int = [str(int(char)) for char in rd]  
    f.close()
    
    n_full_blocks = len(fd_str_int) // L

    array = []
    for i in range(n_full_blocks):
        array.append(' '.join(fd_str_int[i * L : (i + 1) * L]))
        
    array.append(' '.join(fd_str_int[n_full_blocks * L : ]))
    
    compressed = []
    keys = []
    counts = []
    for s in array:
        c, k, count = compress(s)
        compressed.append(c)
        keys.append(k)
        counts.append(count)
    
    import pickle
    
    data = {'keys' : keys, 
           'counts' : counts}
    
    with open(r'keys.pickle', 'wb') as f:
        pickle.dump(data, f)
    
    compressed = ''.join(compressed)
    compressed += '0' * ((len(compressed) // 8 + 1) * 8 - len(compressed))
    
    byte_array = [int(compressed[i * 8 : (i+1) * 8], 2) for i in range (int(len(compressed) / 8))]  
      
    f = open(path_exit, 'wb')
    
    for byte in byte_array:  
        f.write(byte.to_bytes(1,'big'))
        
    return 'Completed!'

In [ ]:
zip_file(r'scale_1200.jpeg', r'zip.txt', 2000)

In [ ]:
def unzip_file(path, path_exit):
    if not os.path.isfile(path):
        raise Exception('Файл не найден!')
        
    # if not os.path.isdir(path_exit):
    #     raise Exception('Укажите путь для выгрузки результаты работы разархиватора!')
        
    with open(path, 'rb') as f:
        data_new = pickle.load(f)
        
    compressed = data_new['compressed']
    keys = data_new['keys']
    
    array = []
    for c, k in zip(compressed, keys):
        text = decompress(c, k)
        array.append(text)
    
    res = []
    for a in array:
        for i in a.split(' '):
            res += i
    
    f = open(path_exit, 'wb')
    for char in fd_int:
        f.write(char.to_bytes(1,'big')) # преобразование int в byte (1 байт на символ - utf8)

    f.close()
    return 'Completed!'

In [ ]:
unzip_file(r'keys.pickle', r'result.jpeg', 2000)